In [ ]:
1+1

In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
plt.rcParams.update({'font.size': 16})
# mpl.rcParams['lines.linewidth'] = 4.

dosave = True
savedir = "plots_1_4_24/systvar/"

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# AMASS = 400
# labelM = "$M_a$ = %i MeV" % AMASS
# label = labelM + "\n$f_a$ = \n$c_\\ell$ ="

In [ ]:
# fs = [
#     "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-CV.df",
#     "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-ind1bin0.df",
#     "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-ind1bin14.df",
#     "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-ind0gainlo.df",
#     "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-ind0gainhi.df",
#     "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-noisehi.df",
# ]

labels = [
    "Nominal Det. Sim.",
    "Middle Ind. Opaque",
    "Middle Ind. Transparent",
    "Front Ind. Gain Low",
    "Front Ind. Gain High",
    "Noise 1.2x",
]

In [ ]:
fs = [
    "/icarus/data/users/gputnam/DMCP2023G/mc-F/F-CohLike_nom_evt.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc-F/F-CohLike_ind0nom_evt.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc-F/F-CohLike_ind1bin0_evt.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc-F/F-CohLike_ind1bin14_evt.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc-F/F-CohLike_noiselhi_evt.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc-F/F-CohLike_sce2x_evt.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc-F/F-CohLike_ind0glo_evt.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc-F/F-CohLike_ind0ghi_evt.df",
]

labels = [
    "Nominal Det. Sim.",
    "Front Ind. WC",
    "Middle Ind. Opaque",
    "Middle Ind. Transparent",
    "Noise 1.1x",
    "SCE 2x",
    "Gain 0.85x",
    "Gain 1.15x",
]

In [ ]:
evtdfs = [pd.read_hdf(f, key="evt") for f in fs]

In [ ]:
hdrdfs = [pd.read_hdf(f, key="hdr") for f in fs]

In [ ]:
GOAL_POT = 3e20

In [ ]:
weights = []
for i in range(len(evtdfs)):
    weights.append(GOAL_POT / np.sum(hdrdfs[i].pot) )

In [ ]:
weights

In [ ]:
def unitdf(df):
    return df.divide(magdf(df), axis=0)

In [ ]:
def recop(trk):
    p = trk.rangeP.p_muon
    p[~TrkInFV(trk.end)] = trk.mcsP.fwdP_muon
    return p

beam2det = np.array([ [0.921035925, 0.022715103, 0.388814672], [0, 0.998297825, -0.058321970], [-0.389477631, 0.053716629, 0.919468161]])

beamorigin = np.array([4.503730e2, 80.153901e2, 795.112945e2])

MUON_MASS = 0.1057 
PION_MASS = 0.13457

BEAMDIR = beam2det.dot(beamorigin) / np.linalg.norm(beam2det.dot(beamorigin)) 
BEAMDIR

In [ ]:
for i in range(len(evtdfs)):
    evtdfs[i]["trunk_mcs_range"] = evtdfs[i].trunk.trk.mcsP.fwdP_muon / evtdfs[i].trunk.trk.rangeP.p_muon
    evtdfs[i]["branch_mcs_range"] = evtdfs[i].branch.trk.mcsP.fwdP_muon / evtdfs[i].branch.trk.rangeP.p_muon
    evtdfs[i]["openangle"] = (180/np.pi)*np.arccos(dotdf(evtdfs[i].trunk.trk.dir, evtdfs[i].branch.trk.dir))
        
    evtdfs[i]["mu_dirx"] = unitdf(evtdfs[i].slc.truth.p0.genp).x
    evtdfs[i]["mu_diry"] = unitdf(evtdfs[i].slc.truth.p0.genp).y
    evtdfs[i]["mu_dirz"] = unitdf(evtdfs[i].slc.truth.p0.genp).z
    evtdfs[i]["mu_phi"] = np.arctan2(evtdfs[i].mu_dirx, evtdfs[i].mu_diry)*180/np.pi
    
    evtdfs[i]["pi_dirx"] = unitdf(evtdfs[i].slc.truth.p1.genp).x
    evtdfs[i]["pi_diry"] = unitdf(evtdfs[i].slc.truth.p1.genp).y
    evtdfs[i]["pi_dirz"] = unitdf(evtdfs[i].slc.truth.p1.genp).z
    evtdfs[i]["pi_phi"] = np.arctan2(evtdfs[i].pi_dirx, evtdfs[i].pi_diry)*180/np.pi
    
    evtdfs[i]["nu_dirx"] = unitdf(evtdfs[i].slc.truth.p0.genp + evtdfs[i].slc.truth.p1.genp).x
    evtdfs[i]["nu_diry"] = unitdf(evtdfs[i].slc.truth.p0.genp + evtdfs[i].slc.truth.p1.genp).y
    evtdfs[i]["nu_dirz"] = unitdf(evtdfs[i].slc.truth.p0.genp + evtdfs[i].slc.truth.p1.genp).z
    evtdfs[i]["nu_phi"] = np.arctan2(evtdfs[i].nu_dirx, evtdfs[i].nu_diry)*180/np.pi
    
    
    Etrunk = np.sqrt(MUON_MASS**2 + recop(evtdfs[i].trunk.trk)**2)
    Ebranch = np.sqrt(MUON_MASS**2 + recop(evtdfs[i].branch.trk)**2)

    evtdfs[i]["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + 
                                             Etrunk*Ebranch - 
                                             recop(evtdfs[i].trunk.trk)*recop(evtdfs[i].branch.trk)*dotdf(evtdfs[i].branch.trk.dir, evtdfs[i].trunk.trk.dir))
    
    scalar_mom = evtdfs[i].trunk.trk.dir.multiply(recop(evtdfs[i].trunk.trk), axis=0) + \
           evtdfs[i].branch.trk.dir.multiply(recop(evtdfs[i].branch.trk), axis=0)
    scalar_dir = scalar_mom.divide(magdf(scalar_mom), axis=0)
    evtdfs[i]["beamangle"] = (180/np.pi)*np.arccos(scalar_dir.x*BEAMDIR[0] + scalar_dir.y*BEAMDIR[1] + scalar_dir.z*BEAMDIR[2])



In [ ]:
bins = np.linspace(0, 1.1, 23) - 0.025
# bins = np.linspace(0.9, 1.1, 6)

for i in range(len(evtdfs)):
    color = "black" if i == 0 else colors[i-1]

    when = (evtdfs[i].trunk.trk.truth.p.start_process == 0) & (np.abs(evtdfs[i].trunk.trk.truth.p.pdg) == 13) &\
        TrkInFV(evtdfs[i].trunk.trk.truth.p.end)

    var = (evtdfs[i].trunk.trk.len) / evtdfs[i].trunk.trk.truth.p.length

    _ = plt.hist(var[when], bins=bins, density=True, label=labels[i], 
                 histtype="step", linewidth=2, color=color)
    
    plt.xlabel("Reco Length / True Length")
    plt.ylabel("Muons (Area Normalized)")
    
plt.legend(loc = "upper left")
# plt.legend()

In [ ]:
def pad(s):
    return tuple(list(s) + [""]*(len(evtdfs[0].columns[0]) - len(s)))

pltvars = [
    ("slc", "truth", "E"),
    ("slc", "truth", "Q2"),
    ("openangle",),
    
    ("mu_dirx",),
    ("mu_diry",),
    ("mu_dirz",),
    ("mu_phi",),

    ("pi_dirx",),
    ("pi_diry",),
    ("pi_dirz",),
    ("pi_phi",),

    ("slc","truth", "p0", "genE"),
    ("slc","truth", "p1", "genE"),

    ("slc", "nuid", "crlongtrkdiry",),
    ("trunk", "trackScore"),
    ("trunk", "trk", "len"),
    
    ("max_shw_len",),
    ("max_othr_trk_len",),
    ("max_othr_chi2_muon",),
    ("min_othr_chi2_proton",),
    
    ("trunk", "trk", "chi2pid", "I2", "chi2_muon"),
    ("trunk", "trk", "chi2pid", "I2", "chi2_proton"),
    ("trunk_mcs_range",),
    ("s_mass",),
    ("beamangle",),
]

pltvars = [pad(v) for v in pltvars]

pltbins = [
    np.linspace(0, 8, 17),
    np.linspace(0, 0.1, 11),
    np.linspace(0, 180, 19),

    np.linspace(-1, 1, 21),
    np.linspace(-1, 1, 21),
    np.linspace(-1, 1, 21),
    np.linspace(-180, 180, 21),
    
    np.linspace(-1, 1, 21),
    np.linspace(-1, 1, 21),
    np.linspace(-1, 1, 21),
    np.linspace(-180, 180, 21),
    
    np.linspace(0, 4, 9),
    np.linspace(0, 4, 9),

    np.linspace(-1, 1, 11),
    np.linspace(0, 1, 11),
    np.linspace(0, 500, 16),
    
    np.linspace(0, 50, 11),
    np.linspace(0, 50, 11),
    np.linspace(0, 60, 11),
    np.linspace(0, 60, 11),
    
    np.linspace(0, 30, 11),
    np.linspace(80, 400, 11),
    np.linspace(0, 2, 11),
    np.linspace(0.2, 0.5, 13),
    np.linspace(0, 20, 11),
]

xlabels = [
    "$\\nu$ Energy [GeV]",
    "Neutrino-Nucleus $Q^2$ [GeV$^2$]", 
    "Muon-Pion Opening Angle",
    
    "Muon $\\hat{x}$",
    "Muon $\\hat{y}$",
    "Muon $\\hat{z}$",
    "Muon $\\phi$",    

    "Pion $\\hat{x}$",
    "Pion $\\hat{y}$",
    "Pion $\\hat{z}$",
    "Pion $\\phi$",    
    
    "Muon Energy [GeV]",
    "Pion Energy [GeV]",
    
    "Pandora CRLongTrkDirY",
    "Trunk Track Pandora Track Score",
    "Trunk Track Length [cm]",
    
    "Maximum Shower Length [cm]",
    "Maximum Third Track Length [cm]",
    "Maximum Other $\\chi^2_\\mu$",
    "Minimum Other $\\chi^2_p$",
    
    "Trunk $\\chi^2_\\mu$",
    "Trunk $\\chi^2_p$",
    "MCS P / Range P",   
    
    "$M_{\\mu\\mu}$ [GeV/$c^2$]",
    "$\\theta_\\mathrm{NuMI}$ [$^\\circ$]"
]

xsave = [
    "nu_E",
    "nu_Q2",
    "openangle",
    "mu_xdir",
    "mu_ydir",
    "mu_zdir",
    "mu_phi",
    "pi_xdir",
    "pi_ydir",
    "pi_zdir",
    "pi_phi",
    "mu_E",
    "pi_E",
    "crlongtrkdiry",
    "trkscore",
    "trklen",
    "max_shw_len",
    "max_thirdtrk_len",
    "max_othr_chi2u",
    "min_othr_chi2p",
    "trunk_chi2u",
    "trunk_chi2p",
    "trunk_range_mcs",
    "scalar_mass",
    "scalar_beamangle"
]

In [ ]:
whens = []
for i in range(len(evtdfs)):
    # this_when = (evtdfs[i].slc.truth.genie_mode == 3) # Require Coh
    this_when = (evtdfs[i].slc.truth.iscc) & (np.abs(evtdfs[i].slc.truth.pdg) == 14) # require numu CC
    this_when = this_when & (np.abs(evtdfs[i].trunk.trk.truth.p.pdg) == 13)
    
    whens.append(this_when)

In [ ]:
savedir = "plots_1_4_24/systvar/"

savedir += "ind0gain_"

In [ ]:
i_dfs = [0] + list(range(1, len(labels)))[5:]
i_dfs
# i_dfs = list(range(len(labels)))

In [ ]:
for i in i_dfs:
    print(labels[i], whens[i].sum()*weights[i], np.sqrt(whens[i].sum())*weights[i])

In [ ]:
for ifig in range(len(pltvars)):
    plt.figure(ifig)
    fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
    fig.subplots_adjust(hspace=0.03)

    var = pltvars[ifig]
    bins = pltbins[ifig]
    centers = (bins[:-1] + bins[1:]) /2

    for i in i_dfs:
        color = "black" if i == 0 else colors[i-1]
        N, _= np.histogram(evtdfs[i][var][whens[i]], bins=bins)
        p0.errorbar(centers, N*weights[i], yerr=np.sqrt(N)*weights[i], 
                     linestyle="none", marker=".", markersize=10, label=labels[i], color=color)
        if i > 0:
            ratio = N*weights[i] / (N0*weights[0])
            p1.errorbar(centers, ratio, 
                        yerr=np.sqrt((ratio**2/N) + (ratio**2/N0)), 
                        color=color, linestyle="none", marker=".")
        else:
            N0 = N

    p0.set_ylabel("# Events / $2\\times 10^{20}$ POT")
    p1.set_ylabel("$\\frac{\\mathrm{Variation}}{\\mathrm{Nominal}}$", fontsize=22)

    p1.set_ylim([0.85, 1.15])
    p1.axhline(1, color="red", linestyle="--")

    p0.legend(loc='upper center', bbox_to_anchor=(0.5, 1.325), ncol=2)
#     p0.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), ncol=2)
    p1.set_xlabel(xlabels[ifig])
    
    if dosave:
        plt.savefig(savedir + "detvar_%s.pdf" % xsave[ifig], bbox_inches="tight")

In [ ]:
# Apply Event Selection

In [ ]:
# OPTIMIZED
MAX_SHW_LEN_CUT = 10
MAX_OTHR_TRK_LEN_CUT = 10
OTHR_CHI2U_CUT = 45
OTHR_CHI2P_CUT = 90

# TRUNK_LEN_CUT = 50
TRUNK_LEN_CUT = 25
BRANCH_LEN_CUT = 25
CHI2U_CUT = 10
CHI2P_CUT = 95
MCS_RANGE_COMP = 1.5
# MCS_RANGE_COMP = 1.8 

STUB_5mm_CUT = 25
STUB_10mm_CUT = 30
STUB_20mm_CUT = 35
STUB_30mm_CUT = 45

OPENANGLE_CUT = 70
UU_ANGLE_CUT = 5

In [ ]:
def cut_all(l):
    l0 = l[0]
    for c in l[1:]:
        l0 = l0 & c
    return l0

def cut_any(l):
    l0 = l[0]
    for c in l[1:]:
        l0 = l0 | c
    return l0

In [ ]:
cutvals = [
    MAX_SHW_LEN_CUT,
    MAX_OTHR_TRK_LEN_CUT,
    OTHR_CHI2U_CUT,
    OTHR_CHI2P_CUT,
    TRUNK_LEN_CUT,
    BRANCH_LEN_CUT,
    CHI2U_CUT,
    CHI2P_CUT,
    CHI2U_CUT,
    CHI2P_CUT,
    MCS_RANGE_COMP,
    MCS_RANGE_COMP,
    STUB_5mm_CUT,
    STUB_10mm_CUT,
    STUB_20mm_CUT,
    STUB_30mm_CUT,
    OPENANGLE_CUT,
    UU_ANGLE_CUT,
]

cut_hi = [
    True,
    True,
    True,
    False,
    False,
    False,
    True,
    False,
    True,
    False,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
]

In [ ]:
passcuts = []

for i in range(len(evtdfs)):
    cutvars = [
        evtdfs[i].max_shw_len_primary,
        evtdfs[i].max_othr_trk_len,
        evtdfs[i].max_othr_chi2_muon,
        evtdfs[i].min_othr_chi2_proton_all,
        evtdfs[i].trunk.trk.len,
        evtdfs[i].branch.trk.len,
        evtdfs[i].trunk.trk.chi2pid.I2.chi2_muon,
        evtdfs[i].trunk.trk.chi2pid.I2.chi2_proton,
        evtdfs[i].branch.trk.chi2pid.I2.chi2_muon,
        evtdfs[i].branch.trk.chi2pid.I2.chi2_proton,
        evtdfs[i].trunk_mcs_range,
        evtdfs[i].branch_mcs_range,
        evtdfs[i].stub.l0_5cm.dedx * evtdfs[i].stub.l0_5cm.length,
        evtdfs[i].stub.l1cm.dedx * evtdfs[i].stub.l1cm.length,
        evtdfs[i].stub.l2cm.dedx * evtdfs[i].stub.l2cm.length,
        evtdfs[i].stub.l3cm.dedx * evtdfs[i].stub.l3cm.length,
        np.arccos(dotdf(evtdfs[i].trunk.trk.dir, evtdfs[i].branch.trk.dir))*180/np.pi,
        evtdfs[i].beamangle,    
    ]

    evtdfs[i]["Truthy"] = True

    wherecut = [
        ~np.isnan(evtdfs[i].max_shw_len_primary),
        ~np.isnan(evtdfs[i].max_othr_trk_len),
        ~np.isnan(evtdfs[i].max_othr_chi2_muon),
        ~np.isnan(evtdfs[i].min_othr_chi2_proton_all),
        evtdfs[i].Truthy,
        evtdfs[i].Truthy,
        evtdfs[i].Truthy,
        evtdfs[i].Truthy,
        evtdfs[i].Truthy,
        evtdfs[i].Truthy,
        evtdfs[i].Truthy,
        evtdfs[i].Truthy,
        ~np.isnan(evtdfs[i].stub.l0_5cm.dedx * evtdfs[i].stub.l0_5cm.length),
        ~np.isnan(evtdfs[i].stub.l1cm.dedx * evtdfs[i].stub.l1cm.length),
        ~np.isnan(evtdfs[i].stub.l2cm.dedx * evtdfs[i].stub.l2cm.length),
        ~np.isnan(evtdfs[i].stub.l3cm.dedx * evtdfs[i].stub.l3cm.length),
        evtdfs[i].Truthy,
        evtdfs[i].Truthy,
    ] 
    cuts = [cutvars[i_cut] < cutvals[i_cut] if cut_hi[i_cut] else cutvars[i_cut] > cutvals[i_cut] 
            for i_cut in range(len(cutvals))]
    cuts = [cuts[i_cut] | ~wherecut[i_cut] for i_cut in range(len(cuts))]
    
    preselect = SlcInFV(evtdfs[i].slc.vertex) & TrkInFV(evtdfs[i].trunk.trk.end) & TrkInFV(evtdfs[i].branch.trk.end)
    
    passcut = preselect & cut_all(cuts)

    passcuts.append(passcut)

In [ ]:
total_evts = np.array([np.sum(passcuts[i])*weights[i] for i in range(len(evtdfs))])
total_evt_var = np.array([np.sum(passcuts[i])*weights[i]**2 for i in range(len(evtdfs))])

ratios =  total_evts/total_evts[0]

ratio_err = np.sqrt(total_evt_var/total_evts[0]**2 + total_evt_var[0]*total_evts**2/total_evts[0]**4)

for l, r, e in zip(labels, ratios, ratio_err):
    print(l, r, "+/-", e)

In [ ]:
# bigger bins post cuts
pltbins = [
    np.linspace(0, 8, 5),
    np.linspace(0, 0.1, 5),
    np.linspace(0, 180, 10),

    np.linspace(-1, 1, 11),
    np.linspace(-1, 1, 11),
    np.linspace(-1, 1, 11),
    np.linspace(-180, 180, 10),
    
    np.linspace(-1, 1, 11),
    np.linspace(-1, 1, 11),
    np.linspace(-1, 1, 11),
    np.linspace(-180, 180, 10),
    
    np.linspace(0, 4, 5),
    np.linspace(0, 4, 5),

    np.linspace(-1, 1, 6),
    np.linspace(0, 1, 6),
    np.linspace(0, 500, 6),
    
    np.linspace(0, 50, 11),
    np.linspace(0, 50, 11),
    np.linspace(0, 60, 11),
    np.linspace(0, 60, 11),
    
    np.linspace(0, 30, 11),
    np.linspace(80, 400, 6),
    np.linspace(0, 1, 6),
    np.linspace(0.2, 0.5, 4),
    np.linspace(0, 20, 11),
]

In [ ]:
for ifig in range(len(pltvars)):
    plt.figure(ifig)
    fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
    fig.subplots_adjust(hspace=0.03)

    var = pltvars[ifig]
    bins = pltbins[ifig]
    centers = (bins[:-1] + bins[1:]) /2

    for i in i_dfs:
        color = "black" if i == 0 else colors[i-1]
        N, _= np.histogram(evtdfs[i][var][passcuts[i]], bins=bins)
        p0.errorbar(centers, N*weights[i], yerr=np.sqrt(N)*weights[i], 
                     linestyle="none", marker=".", markersize=10, label=labels[i], color=color)
        if i > 0:
            ratio = N*weights[i] / (N0*weights[0])
            p1.errorbar(centers, ratio, 
                        yerr=np.sqrt((ratio**2/N) + (ratio**2/N0)), 
                        color=color, linestyle="none", marker=".")
        else:
            N0 = N

    p0.set_ylabel("# Events / $2\\times 10^{20}$ POT")
    p1.set_ylabel("$\\frac{\\mathrm{Variation}}{\\mathrm{Nominal}}$", fontsize=22)

    p1.set_ylim([0.5, 1.5])
    p1.axhline(1, color="red", linestyle="--")

    p0.legend(loc='upper center', bbox_to_anchor=(0.5, 1.325), ncol=2)
#     p0.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), ncol=2)
    p1.set_xlabel(xlabels[ifig])
    
    if dosave:
        plt.savefig(savedir + "signalbox_detvar_%s.pdf" % xsave[ifig], bbox_inches="tight")